## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/TextMining_WordEmbedding

/content/drive/MyDrive/textmining_wordembedding


In [ ]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [ ]:
corpus[:50]

['국내 연구팀, 특정 단백질 증가 현상 찾아내…폐암 조기 진단 키트·신약 개발 기대',
 '정상폐와 폐암에서의 USE1의 조절 메커니즘',
 '[아시아경제 정종오 기자] 폐암을 조기에 진단할 수 있는 생화학 마커를 국내 연구팀이 발견했습니다',
 "폐암환자의 92.5%에서 'USE1' 단백질이 증가돼 있고 이중 13%에서는 USE1 유전자에 돌연변이가 발생해 폐암이 발생한다는 사실을 발견했습니다",
 '폐암을 조기에 진단할 수 있는 새로운 생화학적 지표를 확보했습니다',
 "'USE1'은 생체 내 단백질의 항상성을 조절하는 유비퀴틴 프로테아좀 시스템을 구성하는 효소 중 하나입니다",
 '이번 연구 결과는 폐암에 대한 바이오마커(단백질이나 DNA, RNA, 대사 물질 등을 이용해 몸 안의 변화를 알아낼 수 있는 지표)로 가능해 관심을 모으고 있습니다',
 '폐암은 국제적으로 연간 100만 명이 사망하는 치명적 암입니다',
 '수십 년 동안 진단과 치료 기술이 개발됐는데 특별한 자각증상이 없고 대부분 진행 암 또는 다른 부위에까지도 전이된 상태에서 발견돼 폐암 환자의 완치율은 30% 이하로 낮습니다',
 '폐암을 일찍 발견할 수 있다면 완치율도 높아집니다',
 '조기 진단이 되면 생존율을 80% 이상 획기적으로 늘어납니다',
 '현재 폐암 진단을 위해서는 이미징 방법(X-ray, CT, MRI 등)에 의존도가 높은 편으로 생화학적 지표로 사용될 수 있는 물질은 발견된 예가 적습니다',
 '폐암 진단을 위해 바이오 마커라는 이름으로 제시된 물질들이 존재하는데 여전히 충분한 특이성(specificity)과 민감성(sensitivity)을 나타내지 못하고 있습니다',
 '연구팀은 106명의 폐암환자의 폐암 조직과 정상조직의 단백질의 발현 양을 비교 분석한 결과 폐암조직의 92.5%에서 USE1의 발현양이 증가돼 있음을 확인했습니다',
 '세포와 동물을 사용해 USE1의 발현양이 증가했을 때 폐암을 증식시킴을 확인했습니다',
 '반대로 USE1의 단백질의 발현양이 감소

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

In [ ]:
okt = Okt()

In [ ]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
    return okt.morphs(sentence)

In [ ]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [ ]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec('hyperparameter를 설정해주세요')

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 893


In [ ]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = )

(6506530, 12300500)

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar()

In [ ]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec('hyperparameter를 설정해주세요')

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

In [ ]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

In [ ]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = )

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar()

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [ ]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

4968

In [ ]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_no_pretrained.wv[i] or model_pretrained.wv[i])

In [ ]:
pca = PCA('hyperparameter 설정').fit_transform(vector)
tsne = TSNE('hyperparameter 설정').fit_transform(vector)

In [ ]:
# 시각화해보세요!



# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [ ]:
# t-SNE
tsne = TSNE('hyperparameter 설정').fit_transform(vector)

# 시각화



# 수고하셨습니다!